In [9]:
# Import necessary libraries
from sklearn.linear_model import Ridge, LinearRegression, LogisticRegression
from sklearn.metrics import accuracy_score,log_loss
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV,train_test_split ,StratifiedKFold
from catboost import CatBoostClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor,ExtraTreeClassifier
from sklearn.ensemble import VotingClassifier, BaggingClassifier,BaggingRegressor,RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn.neighbors import *
from sklearn.svm import SVC
from sklearn.pipeline import *
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import warnings
import os

warnings.simplefilter('ignore')

os.chdir('D:\Datasets')

hr = pd.read_csv('Hr_comma_sep.csv')
hr

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,Department,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.10,0.77,6,247,4,0,1,0,sales,low
3,0.92,0.85,5,259,5,0,1,0,sales,low
4,0.89,1.00,5,224,5,0,1,0,sales,low
...,...,...,...,...,...,...,...,...,...,...
14990,0.40,0.57,2,151,3,0,1,0,support,low
14991,0.37,0.48,2,160,3,0,1,0,support,low
14992,0.37,0.53,2,143,3,0,1,0,support,low
14993,0.11,0.96,6,280,4,0,1,0,support,low


In [2]:
hr = pd.read_csv('Hr_comma_sep.csv')
hr

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,Department,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.10,0.77,6,247,4,0,1,0,sales,low
3,0.92,0.85,5,259,5,0,1,0,sales,low
4,0.89,1.00,5,224,5,0,1,0,sales,low
...,...,...,...,...,...,...,...,...,...,...
14990,0.40,0.57,2,151,3,0,1,0,support,low
14991,0.37,0.48,2,160,3,0,1,0,support,low
14992,0.37,0.53,2,143,3,0,1,0,support,low
14993,0.11,0.96,6,280,4,0,1,0,support,low


In [4]:
# Separate features (X) and target variable (y)
x = hr.drop(['left'], axis=1)
y = hr['left']


# catboost wit cataegorical features

In [16]:
# Split the data into training and testing sets
# 'x' and 'y' are the feature and target variables, respectively
# 'test_size=0.3' means 30% of the data will be used for testing
# 'stratify=y' ensures the class distribution in the training and testing sets is the same as the original data
# 'random_state=24' sets the random seed for reproducibility
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, stratify=y, random_state=24)

# Create a Stratified K-Fold cross-validation object
# This will split the data into 5 folds, ensuring the class distribution in each fold is the same as the original data
# 'shuffle=True' ensures the data is shuffled before splitting
# 'random_state=24' sets the random seed for reproducibility
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=24)

# Create an instance of the CatBoost Classifier
# 'random_state=24' sets the random seed for reproducibility
# 'cat_features' specifies the categorical features in the data
cgbm = CatBoostClassifier(random_state=24, cat_features=['Department', 'salary'])

# Fit the CatBoost Classifier to the training data
cgbm.fit(x_train, y_train)

# Use the trained CatBoost Classifier to predict the classes of the test data
y_pred = cgbm.predict(x_test)

# Calculate and print the accuracy score of the predictions
print("Accuracy Score:", accuracy_score(y_test, y_pred))

# Calculate and print the log loss of the predictions
y_pred_prob = cgbm.predict_proba(x_test)[:, 1]
print("Log Loss:", log_loss(y_test, y_pred_prob))

# Define the hyperparameter grid to search over
# 'n_estimators' controls the number of trees in the ensemble
# 'learning_rate' controls the step size of the CatBoost algorithm
# 'max_depth' controls the maximum depth of the individual trees
params = {'n_estimators': [5, 100], 'learning_rate': [0.01, 0.1, 0.7], 'max_depth': [1, 3, 5, 7]}

# Create a GridSearchCV object
# This will perform a grid search to find the best combination of hyperparameters
# 'cv=kfold' uses the Stratified K-Fold cross-validation object
# 'verbose=3' sets the verbosity level to 3, which will print progress updates
# 'scoring='neg_log_loss'' sets the scoring metric to negative log loss (lower is better)
gcv = GridSearchCV(cgbm, param_grid=params, cv=kfold, verbose=3, scoring='neg_log_loss')

# Fit the GridSearchCV object to the data
# This will perform the grid search and find the best hyperparameters
gcv.fit(x, y)

# Print the best parameters found by the GridSearchCV
print("Best Parameters:", gcv.best_params_)

# Print the best score (i.e., the lowest negative log loss) found during the grid search
print("Best Score:", gcv.best_score_)

Learning rate set to 0.028113
0:	learn: 0.6521748	total: 45ms	remaining: 45s
1:	learn: 0.6108150	total: 70.5ms	remaining: 35.2s
2:	learn: 0.5713910	total: 92.5ms	remaining: 30.7s
3:	learn: 0.5320913	total: 115ms	remaining: 28.5s
4:	learn: 0.5001841	total: 135ms	remaining: 26.8s
5:	learn: 0.4684059	total: 156ms	remaining: 25.9s
6:	learn: 0.4390173	total: 178ms	remaining: 25.3s
7:	learn: 0.4134991	total: 201ms	remaining: 24.9s
8:	learn: 0.3904645	total: 225ms	remaining: 24.7s
9:	learn: 0.3706840	total: 246ms	remaining: 24.4s
10:	learn: 0.3509874	total: 268ms	remaining: 24.1s
11:	learn: 0.3328178	total: 289ms	remaining: 23.8s
12:	learn: 0.3167978	total: 310ms	remaining: 23.5s
13:	learn: 0.3023680	total: 330ms	remaining: 23.2s
14:	learn: 0.2893689	total: 351ms	remaining: 23.1s
15:	learn: 0.2766897	total: 371ms	remaining: 22.8s
16:	learn: 0.2649461	total: 395ms	remaining: 22.8s
17:	learn: 0.2543803	total: 422ms	remaining: 23s
18:	learn: 0.2438560	total: 451ms	remaining: 23.3s
19:	learn: 0.2